# Hyperparameter tuning

Here, we're going to create our final model. We will use the tree-based feature selection, and a standard scaler. The detailed model architecture can be found in the 'models' module but to give a summary, it has 4 hidden layers and a softmax output layer.

Like we did in the baseline model notebook, let's import the necessary modules and load in our data.

In [1]:
# so we have access to the Google Drive filesystem
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# necessary imports
import os
import pandas as pd
import numpy as np

# so we can access local modules within Colab
os.chdir('/content/drive/My Drive/auto-age-detector-model')

# feature selection defined functions
from feature_selection import tree_based_feature_selection

# model creation
from models import create_model

# for feature scaling
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


We load in the data.

In [0]:
df_train = pd.read_csv('data/audio_training_data_cleaned.csv').drop(columns=['Unnamed: 0','filename'])
# drop any null values we may have forgotten
df_train = df_train.dropna(how='any',axis=0)
X_train = df_train.drop(columns=['age'],axis=1)
y_train = df_train['age']

Now we use our tree-based feature selector to select the most important features for our model.

In [0]:
X_train,data_transformer = tree_based_feature_selection(X_train,y_train)

We one hot encode the outputs as we did before.

In [0]:
replaced = {'teens':0,'twenties':1,'thirties':2,'fourties':3,'fifties':4,
            'sixties':5,'seventies':6,'eighties':7}

# https://stackoverflow.com/questions/29831489/convert-array-of-indices-to-1-hot-encoded-numpy-array

# need to put one hot encoded in keras model
y_train_ohe = y_train.replace(replaced)
y_train_ohe = np.eye(np.max(y_train_ohe)+1)[y_train_ohe]

Now, we use random search to try out some different hyperparameters. We will focus on finding the right hyperparameters for the dropout and the learning rate. We will use 5-fold cross validation to validate these hyperparameters.

We will be using Adam optimization, as it's generally regarded as the SOA optimizer for deep learning problems.





In [0]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

clf = KerasClassifier(build_fn=create_model,verbose=True)
scaler = StandardScaler()
param_grid = {
    'clf__epochs':[35],
    'clf__dropout':[0.1,0.2],
    'clf__learning_rate':[0.0001,0.001,0.01]
}

pipeline = Pipeline([
    ('preprocess',scaler),
    ('clf',clf)
])

# 5 fold cross validation
rscv = RandomizedSearchCV(pipeline,param_distributions=param_grid,cv=5,n_iter=3)
rscv.fit(X_train,y_train_ohe)

The output is omitted because there were many run-throughs and random searches, but the final model with the best hyperparameters are created in the 'final-model' notebook.